# **DLIP Tutorial - PyTorch**
# CIFAR-10 Classification using PyTorch - Part 2
Y.-K. Kim
(updated 2022. 5. 6) 

===================

The purpose of this tutorial is to learn how to build a simple CNN model for classification of CIFAR-10

You must do: **Tutorial_PyTorch_MNIST_MLP**  before this tutorial

* Part 1:  Create LeNet5 CNN model and Train/Test with opendataset (CIFAR10)
* Part 2-1:  Create a CNN model(VGG-16) 
* Part 2-2:  Create and Train a CNN model(VGG-16) with opendataset(CIFAR-10)

## For CoLab Usage:

1. Download this notebook
2. Then, open in Colab

# Setup Pytorch and Numpy and Device


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {device} device")
if torch.cuda.is_available(): print(f'Device name: {torch.cuda.get_device_name(0)}') 

Using cuda device
Device name: NVIDIA GeForce RTX 3060 Laptop GPU


# Prepare Datasets: Input and Output

In Part 3-1, we will not use dataset for training.  


# (Exercise) Define model - VGG 16

create a class that inherits from nn.Module


* Define the layers of the network in  __init__ function
* Specify Forward network in the **forward function.**


![VGG16.png](https://user-images.githubusercontent.com/23421059/167051297-fc0f379e-bba9-484e-b223-10349d256985.png)

#### Architecture detailed 
![](https://user-images.githubusercontent.com/23421059/167063778-2b1458a8-d7dc-4fc0-acd1-ec0ea51c4a8f.png)

In [7]:
#########################################################
# [EXERCISE] Create VGG-16 architecture (refer to part1)
#########################################################
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()

        self.conv_layers = nn.Sequential(
            # Input 1
            nn.Conv2d(3,64,3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64,64,3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),  #(kernel size, stride size)
            nn.ReLU(),

            # Input 3
            nn.Conv2d(64,128,3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128,128,3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(),

            # Input 5
            nn.Conv2d(128,256,3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256,256,3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(),

            # Input 7
            nn.Conv2d(256,512,3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512,512,3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512,512,3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(),

            # Input 10
            nn.Conv2d(512,512,3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512,512,3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512,512,3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(),
        )

        self.flatten = nn.Flatten()
        
        # Classifier
        self.fc_layers = nn.Sequential(
            # F13
            nn.Linear(7*7*512, 25088),
            nn.ReLU(),
            # F14
            nn.Linear(25088, 4096),
            nn.ReLU(),
            # OUTPUT
            nn.Linear(4096, 1000),
            nn.Softmax(),
        )

    # 위에까진 어떠한 구조로 있는 지 정의

    def forward(self, x):
        # Feature Extraction
        x = self.conv_layers(x)
        # Converting multidimensional data to one dimension for FC operation
        x = self.flatten(x)
        # Classification
        model = self.fc_layers(x)
        return model


model = VGG16().to(device)
print(model)

VGG16(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): ReLU()
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): ReLU()
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU()
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): ReLU()
    (18): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (19): ReLU()
    (20):

Check your model is valid by **summary()** function


In [10]:
from torchsummary import summary
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
              ReLU-6         [-1, 64, 112, 112]               0
            Conv2d-7        [-1, 128, 112, 112]          73,856
              ReLU-8        [-1, 128, 112, 112]               0
            Conv2d-9        [-1, 128, 112, 112]         147,584
             ReLU-10        [-1, 128, 112, 112]               0
        MaxPool2d-11          [-1, 128, 56, 56]               0
             ReLU-12          [-1, 128, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         295,168
             ReLU-14          [-1, 256,